# Example using `julia`

This notebook demonstrates the usage of `toolkitICL` using `julia` to write the configuration file.

In [ ]:
# HDF5 is used for the configuration file.
using HDF5

# helper functions to convert a juia type to its OpenCL equivalent
type_to_name(::Type{Int8}) = "char"
type_to_name(::Type{UInt8}) = "uchar"
type_to_name(::Type{Int16}) = "short"
type_to_name(::Type{UInt16}) = "ushort"
type_to_name(::Type{Int32}) = "int"
type_to_name(::Type{UInt32}) = "uint"
type_to_name(::Type{Int64}) = "long"
type_to_name(::Type{UInt64}) = "ulong"
type_to_name(::Type{Float32}) = "float"
type_to_name(::Type{Float64}) = "double"

# Define an OpenCL kernel and write it to a file. Of course, you can
# write the kernel file in an editor for more complex use cases.
copy_kernel =
"""
#ifdef cl_khr_fp64
  #pragma OPENCL EXTENSION cl_khr_fp64 : enable
#else
  #error \"IEEE-754 double precision not supported by OpenCL implementation.\"
#endif

kernel void copy(global COPYTYPE const* in, global COPYTYPE* out)
{
  const int gid = get_global_id(0);
  out[gid] = in[gid];
}
"""
kernel_url = "copy_kernel_jl.cl"
open(kernel_url, "w") do io
    write(io, copy_kernel)
end

# input parameters
const LENGTH = 32
T = Float64

# generate input/output arrays for OpenCL
input = Vector{T}(0:(LENGTH-1))
output = fill(zero(T), LENGTH)

# write the configuration file
filename = "copy_" * type_to_name(T) * "_test_jl.h5"
h5open(filename, "w") do io
    # compiler arguments for OpenCL
    write(io, "settings/kernel_settings", "-DCOPYTYPE="*type_to_name(T))
    
    # URL of the file containing the kernels
    write(io, "kernel_url", kernel_url)
    # Instead, you could also write(io, "Kernel_Source", copy_kernel).
    
    # kernels to be executed
    write(io, "kernels", ["copy"])

    # OpenCL ranges
    tmp_range = Int32[LENGTH, 1, 1]
    write(io, "settings/global_range", tmp_range)

    tmp_range .= 0
    write(io, "settings/local_range", tmp_range)
    write(io, "settings/range_start", tmp_range)

    # data
    write(io, "data/in", input)
    write(io, "data/out", output)
end


# call toolkitICL
run(`toolkitICL -c $filename`)


# check results
out_filename = "out_" * filename
in_test = h5read(out_filename, "data/in")
vec(in_test) == input || println("Wrong result for 'in'")

out_test = h5read(out_filename, "data/out")
vec(out_test) == input || println("Wrong result for 'out'")

println("Finished successfully.")

In [ ]:
# remove temporary files
rm_if_exists(s) = isfile(s) && rm(s)

rm_if_exists(filename)
rm_if_exists(kernel_url)
rm_if_exists(out_filename)
readdir()